In [2]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
import defillama_utils as dfl
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [3]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

# logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [4]:
# date ranges to build charts for
drange = [0, 1, 7, 30, 90, 180, 365]
# Do we count net flows marked at the lastest token price (1) or the price on each day (0)
# By default, we opt to 1, so that price movement isn't accidentally counted as + or - flow remainder
mark_at_latest_price = 1 #some errors with missing token prices we need to fix first (i.e. rage trade on arbi marks usdc as 0)

trailing_num_days = max(drange)
# print(trailing_num_days)

start_date = date.today()-timedelta(days=trailing_num_days +1)
print(start_date)

# start_date = datetime.strptime('2022-07-13', '%Y-%m-%d').date()



2022-02-07


In [5]:
#get all apps > 5 m tvl
min_tvl = 5_000_000

# if TVL by token is not available, do we fallback on raw TVL (sensitive to token prices)?
is_fallback_on_raw_tvl = True#False

df_df = dfl.get_all_protocol_tvls_by_chain_and_token(min_tvl, is_fallback_on_raw_tvl)

/Users/michaelsilberling/Documents/GitHub/msilb7-crypto-queries/L2 TVL/defillama_utils.py:293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protocols['parentProtocol'] = protocols['parentProtocol'].combine_first(protocols['name'])
/Users/michaelsilberling/Documents/GitHub/msilb7-crypto-queries/L2 TVL/defillama_utils.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )


In [6]:
# display(df_df)
df_df_all = df_df.copy()
df_df_all.head()
# df_df_all[(df_df_all['protocol'] == 'magpie') & (df_df_all['date'] == '2023-01-27')]

,date,token,token_value,usd_value,total_app_tvl,protocol,slug,chain,category,name,parent_protocol
0,2020-12-19 23:00:00,WETH,2272.26688,1.484681e+06,1.484681e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido
1,2020-12-19 23:00:00,MATIC,0,0.000000e+00,1.484681e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido
2,2020-12-20 23:00:00,WETH,4140.77143,2.697598e+06,2.697598e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido
3,2020-12-20 23:00:00,MATIC,0,0.000000e+00,2.697598e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido
4,2020-12-21 23:00:00,WETH,5562.49008,3.410254e+06,3.410254e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido


In [7]:
# display(df_df_all)
df_df_all2 = df_df_all.copy()
# df_df_all2['token_value'] = df_df_all2['token_value'].fillna(0)
df_df_all2['token_value'] = df_df_all2['token_value'].astype('float64')
df_df_all2['usd_value'] = df_df_all2['usd_value'].astype('float64')
# display(df_df_all2)

In [8]:
#create an extra day to handle for tokens dropping to 0

print(df_df_all2.dtypes)

df_df_all_u = df_df_all2.fillna(0)
df_df_shift = df_df_all_u.copy()
df_df_shift['date'] = df_df_shift['date'] + timedelta(days=1)
df_df_shift['token_value'] = 0.0
df_df_shift['usd_value'] = 0.0

#merge back in
df_df_all = pd.concat([df_df_all_u,df_df_shift])

print(df_df_all.dtypes)

# display(df_df_all)
df_df_all = df_df_all[df_df_all['date'] <= pd.to_datetime("today") ]

df_df_all['token_value'] = df_df_all['token_value'].fillna(0)
df_df_all = df_df_all.groupby(['date','token','chain','protocol','name','category','parent_protocol']).sum(['usd_value','token_value'])

# display(df_df_all)
df_df_all = df_df_all.reset_index()
df_df_shift = []
# display(df_df_all)

date               datetime64[ns]
token                      object
token_value               float64
usd_value                 float64
total_app_tvl             float64
protocol                   object
slug                       object
chain                      object
category                   object
name                       object
parent_protocol            object
dtype: object
date               datetime64[ns]
token                      object
token_value               float64
usd_value                 float64
total_app_tvl             float64
protocol                   object
slug                       object
chain                      object
category                   object
name                       object
parent_protocol            object
dtype: object


In [9]:

# df_df_all = pd.concat(df_df_all)
# print(df_df_all[2])
print("done api")
# display(df_df_all[df_df_all['protocol'] == 'velodrome'])
# display(df_df_all)

done api


In [10]:
#filter down a bit so we can do trailing comp w/o doing every row
df_df = df_df_all[df_df_all['date'].dt.date >= start_date-timedelta(days=1) ]

#trailing comp
df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
#now actually filter
df_df = df_df[df_df['date'].dt.date >= start_date ]
# display(df_df[df_df['protocol'] == 'velodrome'])

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48166/588349425.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)


In [11]:
# display(df_df[(df_df['chain'] == 'Arbitrum') & (df_df['protocol'] == 'rage-trade') & (df_df['date'] > '2022-12-01')])

In [12]:
# display(df_df)
# sample = df_df[(df_df['protocol'] == 'uniswap') & (df_df['chain'] == 'Optimism')]
# sample = sample.sort_values(by='date',ascending=False)
# # display(sample)
# sample.to_csv('check_uni_error.csv')

In [13]:
data_df = df_df.copy()
data_df = data_df.sort_values(by='date')
# data_df['token_value'] = data_df['token_value'].replace(0, np.nan) #keep zeroes
# price = usd value / num tokens
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']
data_df['last_price_usd'] = data_df.groupby(['token','protocol', 'chain'])['price_usd'].shift(1)

# If first instnace of token, make sure there's no price diff
data_df['last_price_usd'] = data_df[['last_price_usd', 'price_usd']].bfill(axis=1).iloc[:, 0]
#Forward fill if token drops off
data_df['price_usd'] = data_df[['price_usd','last_price_usd']].bfill(axis=1).iloc[:, 0]

data_df.sample(20)
# display(data_df[data_df['protocol'] == 'velodrome'])

,date,token,chain,protocol,name,category,parent_protocol,token_value,usd_value,total_app_tvl,last_token_value,price_usd,last_price_usd
2916575,2022-04-01,USDC,Optimism,dforce,dForce,Lending,dForce,0.00000,0.000000e+00,1.353016e+06,0.00000,NaN,NaN
3614743,2022-05-28,USDT,Ethereum-borrowed,sashimiswap,SashimiSwap,Dexes,SashimiSwap,3735.76564,3.739501e+03,1.559629e+05,3735.76564,1.001000,1.001000
4491149,2022-07-25,DPT,Binance,biswap,BiSwap,Dexes,BiSwap,664301.77119,1.273261e+03,5.816228e+08,664301.77119,0.001917,0.001917
5955341,2022-10-21,ETH,Binance,belt-finance,Belt Finance,Yield,Belt Finance,3156.11027,4.056643e+06,5.861970e+07,3154.97409,1285.329999,1281.729999
6979151,2022-12-15,RDN,Ethereum,bancor-v2.1,Bancor V2.1,Dexes,Bancor,1093.89288,2.757486e+01,4.003536e+07,1093.89288,0.025208,0.029005
3559134,2022-05-24,UNKNOWN (fantom:0xfc86c61b87e3bc3462b2f2ef8063...,Fantom,spiritswap-amm,SpiritSwap AMM,Dexes,SpiritSwap,0.00000,0.000000e+00,1.200456e+08,0.00000,NaN,NaN
6050596,2022-10-26,MER,Solana-borrowed,solend,Solend,Lending,Solend,6450.14449,9.416559e+01,3.376513e+08,6449.51896,0.014599,0.014516
4701259,2022-08-07,USDT,Ethereum,xtoken,xToken,Services,xToken,0.00000,0.000000e+00,7.279123e+06,0.00000,NaN,NaN
7610951,2023-01-17,WBNB-RPS-CAKE-LP,Binance,bakeryswap,BakerySwap,Dexes,BakerySwap,0.00000,0.000000e+00,1.532747e+07,0.00000,NaN,NaN
6404656,2022-11-14,USDC,IoTeX,relaychain,RelayChain,Bridge,RelayChain,0.00000,0.000000e+00,0.000000e+00,0.00000,NaN,NaN


In [14]:
data_df['token_rank_desc'] = data_df.groupby(['chain','token'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
data_df['token_rank_desc_prot'] = data_df.groupby(['chain','token','protocol'])['date'].\
                            rank(method='dense',ascending=False).astype(int)

data_df['token_rank_desc_prot_gt0'] = data_df.query('token_value > 0')\
                                    .groupby(['chain', 'token', 'protocol'])['date']\
                                    .rank(method='first', ascending=False)

# get latest price either by protocol or in aggregate
# if we don't have a match by protocol, then select in aggregate.

latest_prices_df_raw_prot = data_df[~data_df['price_usd'].isna()][['token','chain','protocol','price_usd']][data_df['token_rank_desc_prot'] ==1]
latest_prices_df_raw = data_df[~data_df['price_usd'].isna()][['token','chain','price_usd']][data_df['token_rank_desc'] ==1]
latest_prices_df_raw_prot_gt0 = data_df[~data_df['price_usd'].isna()][['token','chain','price_usd','protocol']][data_df['token_rank_desc_prot_gt0'] ==1]

latest_prices_df_prot = latest_prices_df_raw_prot.groupby(['token','chain','protocol']).median('price_usd')
latest_prices_df_prot = latest_prices_df_prot.rename(columns={'price_usd':'latest_price_usd_prot'})

latest_prices_df = latest_prices_df_raw.groupby(['token','chain']).median('price_usd')
latest_prices_df = latest_prices_df.rename(columns={'price_usd':'latest_price_usd_raw'})

latest_prices_df_prot_gt0 = latest_prices_df_raw_prot_gt0.groupby(['token','chain','protocol']).median('price_usd')
latest_prices_df_prot_gt0 = latest_prices_df_prot_gt0.rename(columns={'price_usd':'latest_price_usd_prot_gt0'})

latest_prices_df_prot = latest_prices_df_prot.reset_index()
latest_prices_df = latest_prices_df.reset_index()
latest_prices_df_prot_gt0 = latest_prices_df_prot_gt0.reset_index()

prices_df = data_df[['chain','protocol','token']].drop_duplicates()
prices_df = prices_df.merge(latest_prices_df_prot,on=['token','chain','protocol'], how='left')
prices_df = prices_df.merge(latest_prices_df,on=['token','chain'], how='left')
prices_df = prices_df.merge(latest_prices_df_prot_gt0,on=['token','chain','protocol'], how='left')
prices_df['latest_price_usd'] = \
        prices_df['latest_price_usd_prot'].where(prices_df['latest_price_usd_prot'] > 0, \
        prices_df['latest_price_usd_raw'].where(prices_df['latest_price_usd_raw'] > 0, \
        prices_df['latest_price_usd_prot_gt0']))
    # prices_df.loc[prices_df[['latest_price_usd_prot', 'latest_price_usd_raw', 'latest_price_usd_prot_gt0']].first_valid_index()]
# prices_df['latest_price_usd'] = prices_df['latest_price_usd_prot'].combine_first(prices_df['latest_price_usd_raw'])

prices_df = prices_df[['chain','protocol','token','latest_price_usd']]#,'latest_price_usd_prot','latest_price_usd_raw','latest_price_usd_prot_gt0']]

# display(prices_df)
prices_df = prices_df[~prices_df['latest_price_usd'].isna()]


data_df = data_df.merge(prices_df,on=['token','chain','protocol'], how='left')

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48166/3304160164.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  latest_prices_df_raw_prot = data_df[~data_df['price_usd'].isna()][['token','chain','protocol','price_usd']][data_df['token_rank_desc_prot'] ==1]
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48166/3304160164.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  latest_prices_df_raw = data_df[~data_df['price_usd'].isna()][['token','chain','price_usd']][data_df['token_rank_desc'] ==1]
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48166/3304160164.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  latest_prices_df_raw_prot_gt0 = data_df[~data_df['price_usd'].isna()][['token','chain','price_usd','protocol']][data_df['token_rank_desc_prot_gt0'] ==1]


In [15]:
# data_df[(data_df['protocol'] == 'concentrator') & (data_df['token'] == 'FXS') ]
# prices_df[(prices_df['protocol'] == 'concentrator') & (prices_df['token'] == 'FXS')]

In [16]:
# latest_prices_df_prot_gt0[latest_prices_df_prot_gt0['token'] == 'FXS']

In [17]:
data_df.sort_values(by='date',inplace=True)

# net token change
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']
# net token change * current price
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']
# net token change * latest price
data_df['net_dollar_flow_latest_price'] = data_df['net_token_flow'] * data_df['latest_price_usd']


data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar for bad prices
data_df = data_df[~data_df['net_dollar_flow'].isna()]

In [18]:
# Handle for errors where a token price went to zero (i.e. magpie ANKRBNB 2023-01-27)
data_df['net_dollar_flow_latest_price'] = np.where(
    data_df['net_dollar_flow'] == 0, 0, data_df['net_dollar_flow_latest_price']
)

# data_df[(data_df['protocol'] == 'magpie') & (data_df['date'] >= '2023-01-27') ]
# data_df[data_df['protocol'] == 'velodrome']

In [19]:
netdf_df = data_df[['date','protocol','chain','name','category','parent_protocol','net_dollar_flow','usd_value','net_dollar_flow_latest_price']]
netdf_df = netdf_df.fillna(0)
netdf_df = netdf_df.sort_values(by='date',ascending=True)
netdf_df = netdf_df.groupby(['date','protocol','chain','name','category','parent_protocol']).sum(['net_dollar_flow','usd_value','net_dollar_flow_latest_price']) ##agg by app

#usd_value is the TVL on a given day
netdf_df = netdf_df.groupby(['date','protocol','chain','usd_value','name','category','parent_protocol']).sum(['net_dollar_flow','net_dollar_flow_latest_price'])
netdf_df.reset_index(inplace=True)

# netdf_df['cumul_net_dollar_flow'] = netdf_df[['protocol','chain','net_dollar_flow']]\
#                                     .groupby(['protocol','chain']).cumsum()
# netdf_df['cumul_net_dollar_flow_7d'] = netdf_df[['protocol','chain','net_dollar_flow']]\
#                                     .groupby(['protocol','chain'])['net_dollar_flow'].rolling(7, min_periods=1).sum()\
#                                     .reset_index(drop=True)
# netdf_df['cumul_net_dollar_flow_30d'] = netdf_df[['protocol','chain','net_dollar_flow']]\
#                                     .groupby(['protocol','chain'])['net_dollar_flow'].rolling(30, min_periods=1).sum()\
#                                     .reset_index(drop=True)
netdf_df.reset_index(inplace=True)
netdf_df.drop(columns=['index'],inplace=True)
# display(netdf_df[netdf_df['protocol']=='makerdao'])



In [20]:
# tmp = netdf_df[(netdf_df['protocol']=='rage-trade') & (netdf_df['chain']=='Arbitrum') & (netdf_df['date'] > '2022-12-01')]
# tmp.to_csv('check.csv')
# netdf_df[(netdf_df['protocol']=='shibaswap') & (netdf_df['chain']=='Ethereum') & (netdf_df['date'] > '2022-12-01')]

# netdf_df[netdf_df['protocol'] == 'velodrome'].groupby('protocol').sum()

In [21]:
#get latest
netdf_df['rank_desc'] = netdf_df.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
# display(netdf_df[netdf_df['protocol'] == 'lyra'])
netdf_df = netdf_df[  #( netdf_df['rank_desc'] == 1 ) &\
                        (~netdf_df['chain'].str.contains('-borrowed')) &\
                        (~netdf_df['chain'].str.contains('-staking')) &\
                        (~netdf_df['chain'].str.contains('-pool2')) &\
                            (~netdf_df['chain'].str.contains('-treasury')) &\
                        (~( netdf_df['chain'] == 'treasury') ) &\
                        (~( netdf_df['chain'] == 'borrowed') ) &\
                        (~( netdf_df['chain'] == 'staking') ) &\
                            (~( netdf_df['chain'] == 'treasury') ) &\
                        (~( netdf_df['chain'] == 'pool2') ) &\
                        (~( netdf_df['protocol'] == 'polygon-bridge-&-staking') )  &\
                            (~(netdf_df['protocol'].str[-4:] == '-cex') )
#                         & (~( netdf_df['chain'] == 'Ethereum') )
                        ]
# display(netdf_df[netdf_df['protocol']=='makerdao'])

In [22]:
# display(netdf_df)

In [23]:
summary_df = netdf_df.copy()

summary_df = summary_df.sort_values(by='date',ascending=True)

# Mark at latest price if chosen
if mark_at_latest_price == 1:
        summary_df['mark_at_latest_price'] = mark_at_latest_price
        summary_df['net_dollar_flow'] = summary_df['net_dollar_flow_latest_price']
        titleval_append = ' - At Latest Prices'
else:
        titleval_append = ''

# summary_df = summary_df[(summary_df['chain'] == 'Solana') & (summary_df['protocol'] == 'uxd')]
for i in drange:
        if i == 0:
                summary_df['cumul_net_dollar_flow'] = summary_df[['protocol','chain','net_dollar_flow']]\
                                    .groupby(['protocol','chain']).cumsum()
                summary_df['flow_direction'] = np.where(summary_df['cumul_net_dollar_flow']*1.0 >= 0, 1,-1)
                summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])
                #latest price
                
                # display(summary_df)
        else:
                col_str = 'cumul_net_dollar_flow_' + str(i) + 'd'
                tvl_str = 'daily_avg_tvl_' + str(i) + 'd'
                # print(col_str)
                # summary_df[col_str] = summary_df[['protocol','chain','net_dollar_flow']]\
                #                     .groupby(['protocol','chain'])['net_dollar_flow'].transform(lambda x: x.rolling(i, min_periods=1).sum() )
                #chatgpt version
                summary_df[col_str] = summary_df.groupby(['protocol','chain'])['net_dollar_flow']\
                                        .apply(lambda x: x.rolling(i, min_periods=1).sum())

                summary_df[tvl_str] = summary_df[['protocol','chain','usd_value']]\
                                    .groupby(['protocol','chain'])['usd_value'].transform(lambda x: x.rolling(i, min_periods=1).mean() )
                
                summary_df['flow_direction_' + str(i) + 'd'] = np.where(summary_df[col_str]*1.0 >= 0, 1, -1)
                summary_df['abs_cumul_net_dollar_flow_' + str(i) + 'd'] = abs(summary_df[col_str])
                # display(summary_df)
                # display(summary_df[(summary_df['chain'] == 'Optimism') & (summary_df['protocol'] == 'yearn-finance')] )
# display(summary_df[netdf_df['protocol']=='makerdao'])
# display(summary_df[(summary_df['chain'] == 'Optimism') & (summary_df['protocol'] == 'qidao')].iloc[-7: , :15] )
# display(summary_df[(summary_df['protocol']=='stargate') & (summary_df['chain']=='Optimism')])
summary_df['pct_of_tvl'] = 100* summary_df['net_dollar_flow'] / summary_df['usd_value']
final_summary_df = summary_df[(summary_df['rank_desc'] == 1) & (summary_df['date'] >= pd.to_datetime("today") -timedelta(days=7))]
final_summary_df = final_summary_df[final_summary_df['cumul_net_dollar_flow']< 1e20] #weird error handling


os.makedirs('csv_outputs', exist_ok=True)  
final_summary_df.to_csv('csv_outputs/latest_tvl_app_trends.csv')  

# display(summary_df)
for i in drange:
        fig = ''
        if i == 0:
                yval = 'abs_cumul_net_dollar_flow'
                hval = 'cumul_net_dollar_flow'
                cval = 'flow_direction'
                saveval = 'net_app_flows'
                saveval_app = 'net_app_flows_by_app'
                titleval = "App Net Flows Change by Chain -> App - Last " + str(trailing_num_days) + \
                            " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)" + titleval_append
                titleval_app = "App Net Flows Change by App -> Chain - Last " + str(trailing_num_days) + \
                            " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)" + titleval_append
        else:
                yval = 'abs_cumul_net_dollar_flow_' + str(i) +'d'
                hval = 'cumul_net_dollar_flow_' + str(i) +'d'
                cval = 'flow_direction_' + str(i) +'d'
                saveval = 'net_app_flows_' + str(i) +'d'
                saveval_app = 'net_app_flows_by_app_' + str(i) +'d'
                titleval = "App Net Flows Change by Chain -> App - Last " + str(i) + \
                            " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)" + titleval_append
                titleval_app = "App Net Flows Change by App -> Chain - Last " + str(i) + \
                                " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)" + titleval_append
        if is_fallback_on_raw_tvl:
                subtitle = "<br><sup>*For apps where DefiLlama didn't have flows by token, we use their total change in TVL (including token price change)</sup>"
        else:
                subtitle = ""
        # print(yval)
        # print(cval)
        # print(titleval)
        # print(hval)
        fig = px.treemap(final_summary_df[final_summary_df[yval] !=0], \
                 path=[px.Constant("all"), 'chain', 'protocol'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values=yval, color=cval
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='Spectral'
                     , title = titleval + subtitle
                
                , hover_data = [hval]
                )
        
        fig_app = px.treemap(final_summary_df[final_summary_df[yval] !=0], \
                 path=[px.Constant("all"), 'protocol','chain'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values=yval, color=cval
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='Spectral'
                     , title = titleval_app + subtitle
                
                , hover_data = [hval]
                )
        
        fig.update_traces(root_color="lightgrey")
        fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
        fig_app.update_traces(root_color="lightgrey")
        fig_app.update_layout(margin = dict(t=50, l=25, r=25, b=25))

        fig.write_image(prepend + "img_outputs/svg/" + saveval + ".svg") #prepend + 
        fig.write_image(prepend + "img_outputs/png/" + saveval + ".png") #prepend + 
        fig.write_html(prepend + "img_outputs/" + saveval + ".html", include_plotlyjs='cdn')

        fig_app.write_image(prepend + "img_outputs/svg/" + saveval_app + ".svg") #prepend + 
        fig_app.write_image(prepend + "img_outputs/png/" + saveval_app + ".png") #prepend + 
        fig_app.write_html(prepend + "img_outputs/" + saveval_app + ".html", include_plotlyjs='cdn')

        if i == 30:
                fig.show()
# fig.data[0].textinfo = 'label+text+value'
# fig.update_layout(tickprefix = '$')

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48166/2499845005.py:31: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.rolling(i, min_periods=1).sum())
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48166/2499845005.py:31: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group

In [24]:
# display( summary_df[(summary_df['chain'] == 'Arbitrum') & (summary_df['protocol'] == 'rage-trade') & (summary_df['rank_desc'] < 30)][['date','usd_value','protocol','net_dollar_flow','cumul_net_dollar_flow_30d']])

#test sample
# summary_df[summary_df['protocol'] == 'magpie']
summary_df[summary_df['protocol'] == 'velodrome*']

,date,protocol,chain,usd_value,name,category,parent_protocol,net_dollar_flow,net_dollar_flow_latest_price,rank_desc,...,abs_cumul_net_dollar_flow_90d,cumul_net_dollar_flow_180d,daily_avg_tvl_180d,flow_direction_180d,abs_cumul_net_dollar_flow_180d,cumul_net_dollar_flow_365d,daily_avg_tvl_365d,flow_direction_365d,abs_cumul_net_dollar_flow_365d,pct_of_tvl
107534,2022-06-03,velodrome*,Optimism,3.045954e+07,Velodrome,Dexes,Velodrome,1.976815e+06,1.976815e+06,251,...,1.976815e+06,1.976815e+06,3.045954e+07,1,1.976815e+06,1.976815e+06,3.045954e+07,1,1.976815e+06,6.489970
108660,2022-06-04,velodrome*,Optimism,3.919414e+07,Velodrome,Dexes,Velodrome,8.734593e+06,8.734593e+06,250,...,1.071141e+07,1.071141e+07,3.482684e+07,1,1.071141e+07,1.071141e+07,3.482684e+07,1,1.071141e+07,22.285460
109787,2022-06-05,velodrome*,Optimism,3.929542e+07,Velodrome,Dexes,Velodrome,1.012897e+05,1.012897e+05,249,...,1.081270e+07,1.081270e+07,3.631637e+07,1,1.081270e+07,1.081270e+07,3.631637e+07,1,1.081270e+07,0.257765
110919,2022-06-06,velodrome*,Optimism,4.203704e+07,Velodrome,Dexes,Velodrome,2.741616e+06,2.741616e+06,248,...,1.355431e+07,1.355431e+07,3.774654e+07,1,1.355431e+07,1.355431e+07,3.774654e+07,1,1.355431e+07,6.521906
112052,2022-06-07,velodrome*,Optimism,4.871631e+07,Velodrome,Dexes,Velodrome,6.679269e+06,6.679269e+06,247,...,2.023358e+07,2.023358e+07,3.994049e+07,1,2.023358e+07,2.023358e+07,3.994049e+07,1,2.023358e+07,13.710539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469093,2023-02-04,velodrome*,Optimism,1.505152e+08,Velodrome,Dexes,Velodrome,4.064175e+06,4.064175e+06,5,...,6.096472e+07,2.396487e+07,7.860651e+07,1,2.396487e+07,1.220325e+08,7.040909e+07,1,1.220325e+08,2.700176
470808,2023-02-05,velodrome*,Optimism,1.560492e+08,Velodrome,Dexes,Velodrome,5.533983e+06,5.533983e+06,4,...,6.747818e+07,2.722828e+07,7.875778e+07,1,2.722828e+07,1.275665e+08,7.075441e+07,1,1.275665e+08,3.546307
472527,2023-02-06,velodrome*,Optimism,1.575958e+08,Velodrome,Dexes,Velodrome,1.546665e+06,1.546665e+06,3,...,7.006636e+07,3.591572e+07,7.895731e+07,1,3.591572e+07,1.291131e+08,7.110317e+07,1,1.291131e+08,0.981413
474247,2023-02-07,velodrome*,Optimism,1.677307e+08,Velodrome,Dexes,Velodrome,1.013481e+07,1.013481e+07,2,...,9.285640e+07,4.635392e+07,7.921484e+07,1,4.635392e+07,1.392479e+08,7.148968e+07,1,1.392479e+08,6.042314


In [25]:


# test_df= netdf_df[(netdf_df['chain'] == 'Arbitrum') & (netdf_df['protocol'] == 'rage-trade')][['chain','protocol','date','net_dollar_flow','rank_desc']]
# test_df = test_df.sort_values(by='date',ascending=True)
# test_df['test'] = test_df[['protocol','chain','net_dollar_flow']]\
#     .groupby(['protocol','chain'])['net_dollar_flow'].transform(lambda x: x.rolling(30, min_periods=1).sum() )
# display(test_df[test_df['rank_desc'] < 45])
# # display(summary_df[summary_df['protocol']=='makerdao'].iloc[: , :15])

In [26]:

# fig_app = px.treemap(final_summary_df[final_summary_df['abs_cumul_net_dollar_flow'] !=0], \
#                 #  path=[px.Constant("all"), 'chain', 'protocol'], \
# #                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
#                         path=[px.Constant("all"), 'protocol','chain'], \
#                  values='abs_cumul_net_dollar_flow', color='flow_direction'
# #                 ,color_discrete_map={'-1':'red', '1':'green'})
#                 ,color_continuous_scale='Spectral'
#                 , title = "App Net Flows Change by Chain -> App - Last " + str(trailing_num_days) + \
#                             " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)"
#                 ,hover_data=['cumul_net_dollar_flow']
#                 )
# # fig.data[0].textinfo = 'label+text+value'
# fig_app.update_traces(root_color="lightgrey")
# fig_app.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.show()

In [27]:

# fig_app.write_image(prepend + "img_outputs/svg/net_app_flows_by_app.svg") #prepend + 
# fig_app.write_image(prepend + "img_outputs/png/net_app_flows_by_app.png") #prepend + 
# fig_app.write_html(prepend + "img_outputs/net_app_flows_by_app.html", include_plotlyjs='cdn')


In [28]:
# ! jupyter nbconvert --to python total_app_net_flows_async.ipynb